# Paul Unger

## Research question/interests
I'm interested in researching the ratio of how many expensive games get generally positive reviews as opposed to cheaper games to see if the extra money is worth it

In [1]:
import pandas as pd
data_raw = pd.read_csv("../data/raw/steam.csv")
data_raw.head()